In [1]:
import pandas as pd

# 경로 설정
file_path = "gs://codeit-project/hackle/2023-06-17.parquet"
key_path = "/home/leesh/codeit_project/codeit-project-docker/config/key.json"

# GCS에서 parquet 불러오기
df_230617 = pd.read_parquet(file_path, storage_options={"token": key_path})

# 확인
print(df_230617.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4120154 entries, 0 to 4120153
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Asia/Seoul         object 
 1   event_key          object 
 2   session_id         object 
 3   user_id            object 
 4   value              float64
 5   user_properties    object 
 6   hackle_properties  object 
 7   event_properties   object 
 8   id                 object 
 9   device_id          object 
dtypes: float64(1), object(9)
memory usage: 314.3+ MB
None


In [2]:
df_230617.head()

,Asia/Seoul,event_key,session_id,user_id,value,user_properties,hackle_properties,event_properties,id,device_id
0,2023-06-18T00:07:08.014Z,$session_start,5cmi6ko54fPkNcJz6OftyN04nxL2,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[],1f3ac24a-d224-43d3-b2b7-a4d93329d5bc,8b6f9e35-473d-45d7-b8ad-884359609fdd
1,2023-06-18T00:07:11.039Z,$session_end,5cmi6ko54fPkNcJz6OftyN04nxL2,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[],9fdcd5e3-c9cf-4d40-821a-2a7ab78f4e74,8b6f9e35-473d-45d7-b8ad-884359609fdd
2,2023-06-18T00:07:34.065Z,$session_start,WrOTpJzMF0fUed9MlXQggRmyXbw2,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[],94373dd4-7638-4b59-8b2c-fb1a62ebb3f0,4ff18855-0de6-4e51-a00c-7cdfe36bfc64
3,2023-06-18T01:01:12.503Z,$session_start,nn7Kppgo3HfNTFkqiDCJsP2aOVI2,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[],e002e9b9-5cf6-4f0e-b470-f3a6277d42f6,99a6de2d-d222-46ff-81c2-729daff8e688
4,2023-06-18T02:30:47.469Z,$session_start,WrOTpJzMF0fUed9MlXQggRmyXbw2,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",[],e1df00c5-b260-4262-8352-7e4b732977a9,4ff18855-0de6-4e51-a00c-7cdfe36bfc64


In [3]:
df_simple = df_230617.copy()

# 딕셔너리 → Series로 분해 (중요한 컬럼만)
df_simple['language'] = df_simple['hackle_properties'].apply(lambda x: x.get('language'))
df_simple['devicemodel'] = df_simple['hackle_properties'].apply(lambda x: x.get('devicemodel'))


# 1. UTC 기준으로 파싱
df_simple['event_datetime'] = pd.to_datetime(df_simple['Asia/Seoul'], utc=True)

# 2. 한국시간(Asia/Seoul)으로 타임존 변환
df_simple['event_datetime'] = df_simple['event_datetime'].dt.tz_convert('Asia/Seoul')

# 최종 확인용 컬럼만 선택
df_simple = df_simple[['event_datetime','device_id', 'event_key', 'devicemodel', 'language']]

In [4]:
df_simple['event_datetime'] = df_simple['event_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_simple.head()

,event_datetime,device_id,event_key,devicemodel,language
0,2023-06-18 09:07:08,8b6f9e35-473d-45d7-b8ad-884359609fdd,$session_start,"iPhone15,2",ko-KR
1,2023-06-18 09:07:11,8b6f9e35-473d-45d7-b8ad-884359609fdd,$session_end,"iPhone15,2",ko-KR
2,2023-06-18 09:07:34,4ff18855-0de6-4e51-a00c-7cdfe36bfc64,$session_start,"iPhone8,4",ko-KR
3,2023-06-18 10:01:12,99a6de2d-d222-46ff-81c2-729daff8e688,$session_start,"iPad13,1",ko-KR
4,2023-06-18 11:30:47,4ff18855-0de6-4e51-a00c-7cdfe36bfc64,$session_start,"iPhone8,4",ko-KR


In [5]:
df_simple.to_parquet("/home/leesh/codeit_project/codeit-project-docker/parquet/230617.parquet", engine="pyarrow", compression="snappy")